In [9]:
try:
    spark.stop()
except Exception:
    pass

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("Iceberg via REST")
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .config("spark.sql.catalog.rest", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.rest.type", "rest")
    .config("spark.sql.catalog.rest.uri", "http://iceberg-rest:8181")
    .config("spark.sql.catalog.rest.warehouse", "s3://lake/warehouse")
    .config("spark.sql.catalog.rest.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    .config("spark.sql.catalog.rest.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.rest.s3.path-style-access", "true")
    .config("spark.sql.catalog.rest.s3.access-key-id", "admin")
    .config("spark.sql.catalog.rest.s3.secret-access-key", "admin123")
    .config("spark.sql.catalog.rest.s3.region", "us-east-1")
    .getOrCreate()
)

spark

25/11/19 15:04:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/11/19 15:04:30 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [10]:
spark.sql("SHOW NAMESPACES IN rest").show(truncate=False)

+---------+
|namespace|
+---------+
|raw      |
|silver   |
+---------+



In [11]:
spark.sql("SHOW TABLES IN rest.silver").show(truncate=False)

+---------+------------+-----------+
|namespace|tableName   |isTemporary|
+---------+------------+-----------+
|silver   |listings_all|false      |
+---------+------------+-----------+



In [12]:
spark.sql("SELECT COUNT(*) FROM rest.silver.listings_all").show(truncate=False)

+--------+
|count(1)|
+--------+
|532     |
+--------+



In [13]:
silver = spark.sql("SELECT * FROM rest.silver.listings_all")

In [14]:
silver.printSchema()

root
 |-- id: string (nullable = true)
 |-- url: string (nullable = true)
 |-- error: string (nullable = true)
 |-- ingest_ts: timestamp (nullable = true)
 |-- site: string (nullable = false)
 |-- offre: string (nullable = true)
 |-- price: double (nullable = true)
 |-- title: string (nullable = true)
 |-- seller: string (nullable = true)
 |-- published_date: string (nullable = true)
 |-- city: string (nullable = true)
 |-- neighborhood: string (nullable = true)
 |-- property_type: string (nullable = true)
 |-- images: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- equipments: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- description_text: string (nullable = true)
 |-- offre_match: boolean (nullable = true)
 |-- surface_habitable: string (nullable = true)
 |-- caution: string (nullable = true)
 |-- zoning: string (nullable = true)
 |-- type_d_appartement: string (nullable = true)
 |-- standing: string (nullable = true)
 |-- s

In [15]:
silver.limit(1).toPandas()

25/11/19 15:04:43 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

,id,url,error,ingest_ts,site,offre,price,title,seller,published_date,...,disponibilite,salons,features_amenities_json,type_de_bien,surface_de_la_parcelle,type_du_sol,etage_du_bien,annees,orientation,etat
0,55451757,https://www.avito.ma/fr/autre_secteur/terrains...,None,2025-11-19 14:47:40.011,avito,vente,NaN,None,bistrot,2025-11-19 11:47:39,...,None,None,None,None,None,None,None,None,None,None
